In [1]:
import pandas as pd
import sqlite3

## create a connection to the database

In [2]:
db = sqlite3.connect('../data/checking-logs.sqlite.sqlite_copy')

## get the schema of the table test

In [3]:
query = 'PRAGMA table_info(test);'
pd.io.sql.read_sql(query, db)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## get only the first 10 rows of the table test to check what the table looks like

In [4]:
query = 'SELECT * FROM test LIMIT 10;'
pd.io.sql.read_sql(query, db)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
2,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
3,user_17,laba04,2020-04-23 14:24:29.947554,2020-04-18 10:56:55.833899
4,user_18,laba04,2020-04-26 19:48:11.822365,2020-04-26 22:49:29.243278
5,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
7,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
8,user_28,laba04,2020-04-22 21:47:19.707242,2020-05-10 21:07:50.350946
9,user_3,laba04,2020-04-23 20:29:14.054364,2020-05-08 10:53:47.123832


## find among all the users the minimum value of the delta between the first commit of the user and the deadline

In [5]:
query = '''
    SELECT uid, (MIN(deadlines.deadlines - strftime('%s', test.first_commit_ts)) /  3600) as dif FROM test 
    LEFT JOIN deadlines 
    ON labs = labname 
    WHERE labname != 'project1'
    ;'''
df_min = pd.io.sql.read_sql(query, db)
df_min

,uid,dif
0,user_25,2


## do the same thing, but for the maximum, using only one query, the dataframe name is df_max

In [6]:
query = '''
    SELECT uid, (MAX(deadlines.deadlines - strftime('%s', test.first_commit_ts)) /  3600) as dif FROM test 
    LEFT JOIN deadlines 
    ON labs = labname 
    WHERE labname != 'project1'
    ;'''
df_max = pd.io.sql.read_sql(query, db)
df_max

,uid,dif
0,user_30,202


## do the same thing but for the average

In [7]:
query = '''
    SELECT uid, (AVG(deadlines.deadlines - strftime('%s', test.first_commit_ts)) /  3600) as dif FROM test 
    LEFT JOIN deadlines 
    ON labs = labname 
    WHERE labname != 'project1'
    ;'''
df_avg = pd.io.sql.read_sql(query, db)
df_avg

,uid,dif
0,user_1,89.687841


In [8]:
query = '''
    SELECT sub.uid, sub.diff, pageviews
    FROM (
        SELECT uid, (AVG(deadlines.deadlines - strftime('%s', test.first_commit_ts)) /  3600) AS diff
        FROM test
        LEFT JOIN deadlines ON deadlines.labs = test.labname
        WHERE labname != 'project1'
        GROUP BY uid
    ) as sub
    LEFT JOIN
        (SELECT uid, COUNT(*) as pageviews 
        FROM pageviews 
        GROUP BY uid) as pv ON pv.uid = sub.uid
    ;'''
pd.io.sql.read_sql(query, db).to_sql('views_diff', db, if_exists='replace', index=False)
views_diff = pd.io.sql.read_sql(query, db) 
views_diff

,uid,diff,pageviews
0,user_1,65.119778,28
1,user_10,75.242444,89
2,user_14,159.568796,143
3,user_17,62.207667,47
4,user_18,6.368148,3
5,user_19,99.440417,16
6,user_21,96.111181,10
7,user_25,93.474944,179
8,user_28,86.793833,149
9,user_3,105.738222,317


## Use the Pandas method corr() to calculate the correlation coefficient between the number of pageviews and the difference

In [9]:
views_diff.corr()

,diff,pageviews
diff,1.000000,0.279143
pageviews,0.279143,1.000000


## close the connection

In [10]:
db.close()